# Importing dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/datasets/Restaurant reviews.csv')

#Data Exploration and transformation

In [3]:
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [5]:
df.columns

Index(['Restaurant', 'Reviewer', 'Review', 'Rating', 'Metadata', 'Time',
       'Pictures', '7514'],
      dtype='object')

In [8]:
df=df.drop('7514',axis=1)

In [10]:
df=df.drop('Pictures',axis=1)

In [11]:
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37


In [12]:
df.shape

(10000, 6)

columns we need : Rating and Review

In [13]:
df['Rating'].value_counts()

,count
Rating,
5,3832
4,2373
1,1735
3,1193
2,684
4.5,69
3.5,47
2.5,19
1.5,9


In [14]:
df=df[df['Rating'] != 'Like']

In [16]:
df['Rating']=pd.to_numeric(df['Rating'])

In [17]:
def map_sentiments(x):
  if x<3:
    return 0
  else:
    return 1

df['csi']=df['Rating'].apply(map_sentiments)

In [19]:
csi_labels={0:'Negative',1:'Positive'}
df['csi_labels']=df['csi'].map(csi_labels)

In [21]:
df['csi'].value_counts()

,count
csi,
1,7552
0,2447


In [22]:
df.to_csv('/content/drive/MyDrive/datasets/Restaurant_reviews.csv',index=False)

#Reloade the dataset

In [23]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/datasets/Restaurant_reviews.csv')

In [24]:
df.columns

Index(['Restaurant', 'Reviewer', 'Review', 'Rating', 'Metadata', 'Time', 'csi',
       'csi_labels'],
      dtype='object')

# Tokenization

In [25]:
import pandas as pd
import re

In [26]:
def simple_tokenize(text):
  return re.findall(r'\b[a-z]+\b',text.lower())

In [27]:
df['token']=df['Review'].astype(str).apply(simple_tokenize)

In [29]:
df[['Review','token','csi_labels','csi']]

,Review,token,csi_labels,csi
0,"The ambience was good, food was quite good . h...","[the, ambience, was, good, food, was, quite, g...",Positive,1
1,Ambience is too good for a pleasant evening. S...,"[ambience, is, too, good, for, a, pleasant, ev...",Positive,1
2,A must try.. great food great ambience. Thnx f...,"[a, must, try, great, food, great, ambience, t...",Positive,1
3,Soumen das and Arun was a great guy. Only beca...,"[soumen, das, and, arun, was, a, great, guy, o...",Positive,1
4,Food is good.we ordered Kodi drumsticks and ba...,"[food, is, good, we, ordered, kodi, drumsticks...",Positive,1
...,...,...,...,...
9994,Madhumathi Mahajan Well to start with nice cou...,"[madhumathi, mahajan, well, to, start, with, n...",Positive,1
9995,This place has never disappointed us.. The foo...,"[this, place, has, never, disappointed, us, th...",Positive,1
9996,"Bad rating is mainly because of ""Chicken Bone ...","[bad, rating, is, mainly, because, of, chicken...",Negative,0
9997,I personally love and prefer Chinese Food. Had...,"[i, personally, love, and, prefer, chinese, fo...",Positive,1


In [30]:
text='The food was GREAT! 100'

In [31]:
simple_tokenize(text)

['the', 'food', 'was', 'great']

#Removing stop words

In [32]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
stop_words=set(stopwords.words('english'))

In [34]:
def remove_stopwords(token):
  return [word for word in token if word.lower() not in stop_words]

In [35]:
df['tokens_no_stopwords']=df['token'].apply(remove_stopwords)

In [36]:
df.columns

Index(['Restaurant', 'Reviewer', 'Review', 'Rating', 'Metadata', 'Time', 'csi',
       'csi_labels', 'token', 'tokens_no_stopwords'],
      dtype='object')

In [37]:
df[['Review','token','csi_labels','csi','tokens_no_stopwords']]

,Review,token,csi_labels,csi,tokens_no_stopwords
0,"The ambience was good, food was quite good . h...","[the, ambience, was, good, food, was, quite, g...",Positive,1,"[ambience, good, food, quite, good, saturday, ..."
1,Ambience is too good for a pleasant evening. S...,"[ambience, is, too, good, for, a, pleasant, ev...",Positive,1,"[ambience, good, pleasant, evening, service, p..."
2,A must try.. great food great ambience. Thnx f...,"[a, must, try, great, food, great, ambience, t...",Positive,1,"[must, try, great, food, great, ambience, thnx..."
3,Soumen das and Arun was a great guy. Only beca...,"[soumen, das, and, arun, was, a, great, guy, o...",Positive,1,"[soumen, das, arun, great, guy, behavior, sinc..."
4,Food is good.we ordered Kodi drumsticks and ba...,"[food, is, good, we, ordered, kodi, drumsticks...",Positive,1,"[food, good, ordered, kodi, drumsticks, basket..."
...,...,...,...,...,...
9994,Madhumathi Mahajan Well to start with nice cou...,"[madhumathi, mahajan, well, to, start, with, n...",Positive,1,"[madhumathi, mahajan, well, start, nice, court..."
9995,This place has never disappointed us.. The foo...,"[this, place, has, never, disappointed, us, th...",Positive,1,"[place, never, disappointed, us, food, courteo..."
9996,"Bad rating is mainly because of ""Chicken Bone ...","[bad, rating, is, mainly, because, of, chicken...",Negative,0,"[bad, rating, mainly, chicken, bone, found, ve..."
9997,I personally love and prefer Chinese Food. Had...,"[i, personally, love, and, prefer, chinese, fo...",Positive,1,"[personally, love, prefer, chinese, food, coup..."


#Text to Vectors (TF-IDF)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
df['text_cleaned']=df['tokens_no_stopwords'].apply(lambda x:' '.join(x))

In [42]:
tfidf_vectorizer=TfidfVectorizer(max_features=2500)

In [43]:
X_tfidf=tfidf_vectorizer.fit_transform(df['text_cleaned']) #imp

In [44]:
df.columns

Index(['Restaurant', 'Reviewer', 'Review', 'Rating', 'Metadata', 'Time', 'csi',
       'csi_labels', 'token', 'tokens_no_stopwords', 'text_cleaned'],
      dtype='object')

In [46]:
df[['csi_labels','csi','tokens_no_stopwords','text_cleaned']]

,csi_labels,csi,tokens_no_stopwords,text_cleaned
0,Positive,1,"[ambience, good, food, quite, good, saturday, ...",ambience good food quite good saturday lunch c...
1,Positive,1,"[ambience, good, pleasant, evening, service, p...",ambience good pleasant evening service prompt ...
2,Positive,1,"[must, try, great, food, great, ambience, thnx...",must try great food great ambience thnx servic...
3,Positive,1,"[soumen, das, arun, great, guy, behavior, sinc...",soumen das arun great guy behavior sincerety g...
4,Positive,1,"[food, good, ordered, kodi, drumsticks, basket...",food good ordered kodi drumsticks basket mutto...
...,...,...,...,...
9994,Positive,1,"[madhumathi, mahajan, well, start, nice, court...",madhumathi mahajan well start nice courteous s...
9995,Positive,1,"[place, never, disappointed, us, food, courteo...",place never disappointed us food courteous sta...
9996,Negative,0,"[bad, rating, mainly, chicken, bone, found, ve...",bad rating mainly chicken bone found veg food ...
9997,Positive,1,"[personally, love, prefer, chinese, food, coup...",personally love prefer chinese food couple tim...


#MLDL

In [64]:
#input

X=X_tfidf
y=df['csi_labels']

In [65]:
#logistic reg

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [66]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [67]:
from sklearn.metrics import accuracy_score,classification_report

y_pred=model.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8945
              precision    recall  f1-score   support

    Negative       0.87      0.69      0.77       507
    Positive       0.90      0.96      0.93      1493

    accuracy                           0.89      2000
   macro avg       0.88      0.83      0.85      2000
weighted avg       0.89      0.89      0.89      2000



#Prediction

In [75]:
def predict_review(text):
  tokens=simple_tokenize(text)
  tokens_no_stopwords=remove_stopwords(tokens)
  text_cleaned=' '.join(tokens_no_stopwords)
  X_tfidf=tfidf_vectorizer.transform([text_cleaned])
  y_pred=model.predict(X_tfidf)
  return y_pred[0]

In [76]:
user_input = input("Enter a customer review: ")
sentiment = predict_review(user_input)
print(f"Predicted Sentiment: {sentiment}")


Enter a customer review: good product, must buy!
Predicted Sentiment: Positive


In [78]:
import joblib

joblib.dump(model, '/content/drive/MyDrive/deepcsat_RReview.pkl')

['/content/drive/MyDrive/deepcsat_RReview.pkl']

In [81]:
joblib.dump(tfidf_vectorizer, '/content/drive/MyDrive/vectorizer.pkl')

['/content/drive/MyDrive/vectorizer.pkl']